In [1]:
import PEUQSE
import numpy as np 
import pandas as pd 
import yaml
import math
import rms_simulation


In [2]:
def initial_val_unc(kinetics, num): 
    """ 
    return the initial value, an estimate of parameter uncertainty, and upper
     as a tuple
    A-factors: if not sticking coeffient, +/- 1, return LogA. else, +/- 0.5
    Ea: +/- 0.3 eV. 

    Limits: 
    A (upper):  - 1 for sticking coefficient, 1e25 for others
    A (lower):  - 0 for sticking coefficient, 0 for others
    E (upper):  - 400 kJ/mol
    E (lower):  - 0 kj/mol
    """ 
    # logA if it is not sticking coefficient
    if kinetics["A"] > 1: 
        A = math.log10(kinetics["A"])
        A_unc = 1
        A_lb = 0
        A_ub = 1e25
        label_A = f"A_log_{num}"
    elif kinetics["A"] <= 1: 
        A = kinetics["A"]
        A_unc = 1
        A_lb = 0
        A_ub = 1
        label_A = f"A_stick_{num}"
    
    E = kinetics["Ea"]
    E_unc = 30000
    E_lb = 0
    E_ub = 400000
    label_E = f"E_{num}"
    
    return (A, E), (A_unc, E_unc), (A_lb, E_lb), (A_ub, E_ub), (label_A, label_E)

In [3]:
# get the input parameter list. we need: 
# for A, Ea, etc: 
# 1. a list of the initial values
# 2. a list of the uncertainties

# for expt data
# 3. x_data: the pressure, temp, etc for each experiment
# 4. uncertainty data for the x_data
# 4. y_data: the output mole 
# fractions for CO, CO2, etc. 
# 6. Uncertainty data for the y_data
rms_file = "../baseline/rms/chem53.rms"
with open(rms_file, "r") as f: 
    rms_mech = yaml.load(f, Loader=yaml.FullLoader)

initial_values = []
uncertainties = []
lower_bounds = []
upper_bounds = []
labels = []

for num, rxn in enumerate(rms_mech["Reactions"]):
    kin = rxn["kinetics"]
    new_kin = initial_val_unc(kin, num)
    initial_values.extend(new_kin[0])
    uncertainties.extend(new_kin[1])
    lower_bounds.extend(new_kin[2])
    upper_bounds.extend(new_kin[3])
    labels.extend(new_kin[4])


# make a new yaml for these lists 
yaml_dict = {
    "initial_values": initial_values,
    "uncertainties": uncertainties,
    "lower_bounds": lower_bounds,
    "upper_bounds": upper_bounds,
    "labels": labels
}
with open("rms_initial.yaml", "w") as f: 
    yaml.dump(yaml_dict, f)

print(len(yaml_dict["initial_values"]))


1348


In [5]:
# now get experimental data
expt_yaml = "../gua_cantera/all_experiments_reorg_sbr.yaml"

# get all experiments that have "use_for_opt" = True and Temp < 518L

with open(expt_yaml, "r") as f:
    expt_list = yaml.load(f, Loader=yaml.FullLoader)

new_expt_yaml = []
for expt in expt_list:
    if "use_for_opt" in expt.keys():
        if expt["use_for_opt"] == True and expt["temperature"] < 518:
            new_expt_yaml.append(expt)

# save the new yaml
new_expt_file = "./expt_data.yaml"
with open(new_expt_file, "w") as f:
    yaml.dump(new_expt_yaml, f)


# load the experimental data yaml so we get a dict of lists for each parameter
expt_data = rms_simulation.repackage_yaml(new_expt_file)
print(len(expt_data['catalyst_area']))

# save the yaml file 
with open("expt_data.yaml", "w") as f:
    yaml.dump(expt_data, f)




74


dict_keys(['catalyst_area', 'experiment_type', 'expt_name', 'output_CH3OH', 'output_H2O', 'pressure', 'species_CO', 'species_CO2', 'species_H2', 'species_H2O', 'species_out_CH3OH', 'species_out_CO', 'species_out_CO2', 'species_out_H2', 'species_out_H2O', 'temperature', 'use_for_opt', 'volume', 'volume_flowrate'])

#### generate uncertainty yaml_file

In [40]:
unc_yaml = {}
unc_yaml["catalyst_area"] = len(expt_data["catalyst_area"])*[7]
unc_yaml["pressure"] = len(expt_data["pressure"])*[1e3]
unc_yaml["species_CO"] = list(np.array(expt_data["species_CO"])*0.01)
unc_yaml["species_CO2"] = list(np.array(expt_data["species_CO2"])*0.01)
unc_yaml["species_H2"] = list(np.array(expt_data["species_H2"])*0.01)
unc_yaml["species_H2O"] = list(np.array(expt_data["species_H2O"])*0.01)
unc_yaml["species_out_CH3OH"] = list(np.array(expt_data["species_out_CH3OH"])*0.01)
unc_yaml["species_out_H2O"] = list(np.array(expt_data["species_out_H2O"])*0.01)
unc_yaml["species_out_H2"] = list(np.array(expt_data["species_out_H2"])*0.01)
unc_yaml["species_out_CO"] = list(np.array(expt_data["species_out_CO"])*0.01)
unc_yaml["species_out_CO2"] = list(np.array(expt_data["species_out_CO2"])*0.01)
unc_yaml["temperature"] = len(expt_data["temperature"])*[0.05]





[0.065,
 0.065,
 0.065,
 0.065,
 0.065,
 0.065,
 0.065,
 0.065,
 0.065,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.053,
 0.22,
 0.22,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.179,
 0.179,
 0.179,
 0.179,
 0.179,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092,
 0.092]

In [23]:
n_reactions = len(rms_mech["Reactions"])

674

In [25]:
rxn_nums = np.arange(n_reactions)
rxn_nums[0], rxn_nums[-1]


(0, 673)

In [29]:
# unpack the peuquse parametersfor use in modifying sim input
input_a_ea = dict(zip(yaml_dict["labels"], yaml_dict["initial_values"]))
a_list 
for key,value in input_a_ea.items():
    rxn = key.split("_")[-1]
    input_a_ea_rxn[rxn] = value


#     print(val)


{'A_stick_0': 0.046,
 'E_0': 0.0,
 'A_log_1': 15.868644438394826,
 'E_1': 77300.0,
 'A_stick_2': 0.75,
 'E_2': 0.0,
 'A_log_3': 15.060697840353612,
 'E_3': 101400.0,
 'A_log_4': 16.0,
 'E_4': 90500.0,
 'A_log_5': 11.0,
 'E_5': 169500.0,
 'A_stick_6': 0.005,
 'E_6': 0.0,
 'A_log_7': 17.568201724066995,
 'E_7': 117600.0,
 'A_log_8': 15.0,
 'E_8': 38700.0,
 'A_log_9': 3.4608978427565478,
 'E_9': 343506.0,
 'A_stick_10': 2.54e-07,
 'E_10': 223258.0,
 'A_log_11': 3.1789769472931697,
 'E_11': 309198.0,
 'A_stick_12': 1.27e-07,
 'E_12': 223258.0,
 'A_stick_13': 1e-05,
 'E_13': 68660.0,
 'A_log_14': 13.0,
 'E_14': 50000.0,
 'A_stick_15': 1e-05,
 'E_15': 68660.0,
 'A_stick_16': 0.1,
 'E_16': 0.0,
 'A_stick_17': 0.1,
 'E_17': 0.0,
 'A_stick_18': 5e-06,
 'E_18': 68660.0,
 'A_stick_19': 5e-06,
 'E_19': 68660.0,
 'A_stick_20': 5e-06,
 'E_20': 68660.0,
 'A_stick_21': 0.1,
 'E_21': 0.0,
 'A_stick_22': 0.02,
 'E_22': 41840.0,
 'A_stick_23': 0.2,
 'E_23': 146749.00825292533,
 'A_stick_24': 0.2,
 'E_24'

In [32]:
dicty = { 'A_stick_1': 0.1,
 'E_16': 0.0,
 'A_stick_17': 0.1,
 'E_17': 0.0,}

for key in dicty.keys():
    print(key.split("_")[-1])

1
16
17
17
